In [162]:
import pandas as pd
import yahoo_fantasy_api as yfa
from yahoo_oauth import OAuth2
import gspread

In [164]:
# # Google Sheets setup
# scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
#          "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
# creds = ServiceAccountCredentials.from_json_keyfile_name('GoogleApi.json', scope)
# client = gspread.authorize(creds)

# # Export DataFrame to Google Sheets
# sheet_name = "Fantasy League Stats"
# spreadsheet = client.create(sheet_name)
# worksheet = spreadsheet.sheet1
# worksheet.update([df.columns.values.tolist()] + df.values.tolist())  # Upload the DataFrame data


In [165]:
#Auth
sc = OAuth2(None, None, from_file="YahooAuth.json")

[2024-11-11 16:01:07,454 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2024-11-11 16:01:07,477 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 315657.19510245323
[2024-11-11 16:01:07,479 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN HAS EXPIRED
[2024-11-11 16:01:07,481 DEBUG] [yahoo_oauth.oauth.refresh_access_token] REFRESHING TOKEN


In [166]:
#Get Game object
gm = yfa.Game(sc, 'nba')

In [167]:
leagues = gm.league_ids()
league = gm.to_league(leagues[-1])

In [168]:
data = league.matchups(week=3)['fantasy_content']['league'][1]['scoreboard']['0']['matchups']
data

{'0': {'matchup': {'week': '3',
   'week_start': '2024-11-04',
   'week_end': '2024-11-10',
   'status': 'postevent',
   'is_playoffs': '0',
   'is_consolation': '0',
   'is_tied': 0,
   'winner_team_key': '454.l.74462.t.1',
   'stat_winners': [{'stat_winner': {'stat_id': '5',
      'winner_team_key': '454.l.74462.t.6'}},
    {'stat_winner': {'stat_id': '8', 'winner_team_key': '454.l.74462.t.1'}},
    {'stat_winner': {'stat_id': '10', 'winner_team_key': '454.l.74462.t.1'}},
    {'stat_winner': {'stat_id': '12', 'winner_team_key': '454.l.74462.t.6'}},
    {'stat_winner': {'stat_id': '15', 'winner_team_key': '454.l.74462.t.1'}},
    {'stat_winner': {'stat_id': '16', 'winner_team_key': '454.l.74462.t.6'}},
    {'stat_winner': {'stat_id': '17', 'winner_team_key': '454.l.74462.t.1'}},
    {'stat_winner': {'stat_id': '18', 'winner_team_key': '454.l.74462.t.1'}},
    {'stat_winner': {'stat_id': '19', 'winner_team_key': '454.l.74462.t.1'}}],
   '0': {'teams': {'0': {'team': [[{'team_key': '454

In [169]:
#Dict of categories
categories = {
    "5": "FG%",
    "8": "FT%",
    "10": "3PM",
    "12": "PTS",
    "15": "REB",
    "16": "AST",
    "17": "STL",
    "18": "BLK",
    "19": "TO"
}

In [170]:
# Initialize a list to collect each team’s data
teams_data = []

# Iterate over each matchup and team to extract stats
for matchup_id, matchup_data in data.items():
    # Check that matchup_data is a dictionary
    if not isinstance(matchup_data, dict):
        continue  # Skip any non-dictionary entries

    matchup = matchup_data.get('matchup')
    if not matchup:
        continue
    
    # Go through each team in the matchup, ensure '0' key exists and is a dictionary
    teams = matchup.get('0', {}).get('teams', {})
    if not isinstance(teams, dict):
        continue

    for team_key, team_data in teams.items():
        # Check that team_data is a dictionary
        if not isinstance(team_data, dict) or 'team' not in team_data:
            continue

        # Ensure team_data['team'] is a list
        if not isinstance(team_data['team'], list) or len(team_data['team']) < 2:
            continue

        team = team_data['team'][0]  # Access the team list
        team_name = team[2]['name']  # Extract team name
        
        # Initialize the row with the team name
        team_row = {'Manager': team_name}

        # Fetch team stats for the week
        stats = team_data['team'][1].get('team_stats', {}).get('stats', [])
        
        # Extract relevant stats by stat_id
        for stat in stats:
            stat_id = stat['stat']['stat_id']
            if stat_id in categories:
                stat_name = categories[stat_id]
                team_row[stat_name] = stat['stat']['value']

        # Get the games played and remaining games
        games_info = team_data['team'][1].get('team_remaining_games', {}).get('total', {})
        games_played = games_info.get('completed_games', 0)
        remaining_games = games_info.get('remaining_games', 0)
        
        # Calculate total games and format as "games played / total games"
        total_games = games_played + remaining_games
        team_row['Games Played'] = f"{games_played}/{total_games}"
        team_row['Games Remaining'] = remaining_games

        # Add team row to the list
        teams_data.append(team_row)


In [171]:
teams_data = pd.DataFrame(teams_data)
teams_data

,Manager,FG%,FT%,3PM,PTS,REB,AST,STL,BLK,TO,Games Played,Games Remaining
0,Ducksnation,.427,.802,94,655,261,104,60,51,81,45/45,0
1,Big Gradey Energy,.499,.787,64,675,228,206,43,32,96,44/44,0
2,GG no re,.494,.814,83,758,259,152,30,28,84,47/47,0
3,CarMelo Ball,.498,.781,55,599,216,148,39,17,65,44/44,0
4,Goat Club,.507,.850,65,777,321,189,35,24,81,43/43,0
5,Felix’s team,.498,.815,105,806,193,170,50,28,80,44/44,0
6,The Good Gordons,.456,.773,70,677,301,233,45,29,101,46/46,0
7,Bog's Tots,.469,.785,56,772,286,174,51,27,104,45/45,0
8,Thomas's Team,.475,.789,100,853,253,172,49,23,99,45/45,0
9,Fab's Fascinating Team,.483,.840,85,847,268,174,29,36,113,43/43,0
